In [1]:
from util.run import solve

This is a simple encoding of the monty hall problem, where all the decisions are done at the same time.

In [2]:
simple_program_str = """
% There are three doors.
door(1..3).

% There are three actors in the story.
actor(gamemaster). % Chooses where to hide the prize.
actor(player).     % Chooses the initial door and if they switch, after Monty opened a door.
actor(monty).      % Chooses to open a door that does not have the prize.

% Each actor chooses exactly one door.
{ doorchosen_by(D, A) : door(D) } = 1 :- actor(A).

% Monty may only choose a door that was not chosen by the gamemaster
:- doorchosen_by(1, monty), doorchosen_by(1, gamemaster).
:- doorchosen_by(2, monty), doorchosen_by(2, gamemaster).
:- doorchosen_by(3, monty), doorchosen_by(3, gamemaster).

% Monty may only choose a door that was not chosen by the player
:- doorchosen_by(1, monty), doorchosen_by(1, player).
:- doorchosen_by(2, monty), doorchosen_by(2, player).
:- doorchosen_by(3, monty), doorchosen_by(3, player).

% The player may switch or stay (not switch).
{switch; -switch} = 1.

% The player has won if they initially picked the right door and chose not to switch.
won :- door(D),                      
  doorchosen_by(D, gamemaster), doorchosen_by(D, player),
  not switch.
  
% The player has won if they initially picked the wrong door and chose to switch.
won :- door(D1), door(D2), D1 != D2,
  doorchosen_by(D1, gamemaster), doorchosen_by(D2, player),
  switch.

% Not winning means losing.
lost :- not won.

%:- lost.
"""
solve(simple_program_str)

Answer  1: { door(1) door(2) door(3) doorchosen_by(2,monty) doorchosen_by(3,gamemaster) doorchosen_by(3,player) actor(gamemaster) actor(player) actor(monty) -switch won }.
Answer  2: { door(1) door(2) door(3) doorchosen_by(1,monty) doorchosen_by(3,gamemaster) doorchosen_by(3,player) actor(gamemaster) actor(player) actor(monty) -switch won }.
Answer  3: { door(1) door(2) door(3) doorchosen_by(1,gamemaster) doorchosen_by(2,monty) doorchosen_by(3,player) actor(gamemaster) actor(player) actor(monty) -switch lost }.
Answer  4: { door(1) door(2) door(3) doorchosen_by(1,monty) doorchosen_by(2,gamemaster) doorchosen_by(3,player) actor(gamemaster) actor(player) actor(monty) -switch lost }.
Answer  5: { door(1) door(2) door(3) doorchosen_by(1,monty) doorchosen_by(2,gamemaster) doorchosen_by(2,player) actor(gamemaster) actor(player) actor(monty) -switch won }.
Answer  6: { door(1) door(2) door(3) doorchosen_by(2,gamemaster) doorchosen_by(2,player) doorchosen_by(3,monty) actor(gamemaster) actor(pl

The following code blocks describe a more sophisticated encoding, with an explicit encoding of time.

In [3]:
event_calculus_axioms = """
% as described in https://arxiv.org/abs/2106.14566 page 14 (explanaitions) and page 17 (code)

mintime(T) :- time(T), not time(T-1).
maxtime(T) :- time(T), not time(T+1).


%% BEC1
stoppedIn(T1, F, T2) :-
  time(T1), time(T2), time(T),
  fluent(F),
  event(E),
  T1 < T, T < T2,
  terminates(E, F, T),
  happens(E, T).
  
stoppedIn(T1, F, T2) :-
  time(T1), time(T2), time(T),
  fluent(F),
  event(E),
  T1 < T, T < T2,
  releases(E, F, T),
  happens(E, T).


%% BEC2
startedIn(T1, F, T2) :-
  time(T1), time(T2), time(T),
  fluent(F),
  event(E),
  T1 < T, T < T2,
  initiates(E, F, T),
  happens(E, T).

startedIn(T1, F, T2) :-
  time(T1), time(T2), time(T),
  fluent(F),
  event(E),
  T1 < T, T < T2,
  releases(E, F, T),
  happens(E, T).


%% BEC3
holdsAt(F2, T2) :-
  time(T1), time(T2), time(T),
  fluent(F1), fluent(F2),
  trajectory(F1,T1,F2,T2),
  not stoppedIn(T1,F1,T2).


%% BEC4
holdsAt(F, T) :-
  time(T), mintime(MT),
  fluent(F),
  initiallyP(F),
  not stoppedIn(MT, F, T).
  
-holdsAt(F, T) :-
  time(T), mintime(MT),
  fluent(F),
  initiallyN(F),
  not startedIn(MT, F, T).


%% BEC6
holdsAt(F,T2) :-
  time(T1), time(T2),
  fluent(F),
  event(E),
  T1 < T2,
  initiates (E,F,T1),
  happens(E,T1),
  not stoppedIn (T1,F,T2).


%% BEC7
-holdsAt(F,T2) :-
  time(T1), time(T2),
  fluent(F),
  event(E),
  T1 < T2,
  terminates(E,F,T1),
  happens(E,T1),
  not startedIn (T1,F,T2).


%% Consistency
:- -holdsAt(F,T), holdsAt(F,T).
"""

In [4]:
monty = """

% HELPERS
door(1..3).
actor(player).
actor(gamemaster).
actor(monty).


% TIME
time(1..6).

% FLUENTS
fluent(prizeisbehind(D)) :- door(D).
fluent(picked(D)) :- door(D).
fluent(opened(D)) :- door(D).
fluent(switched).

% EVENTS
event(choosesdoor_by(D, A)) :- door(D), actor(A).
event(switch).
event(stay).

% STORY

% initially no door is chosen
initiallyN(prizeisbehind(D)) :- door(D).
initiallyN(picked(D)) :- door(D).
initiallyN(openend(D)) :- door(D).

{ happens(choosesdoor_by(D, gamemaster), 2)                                 : door(D) } = 1.
initiates(choosesdoor_by(D, gamemaster), prizeisbehind(D), T) :- time(T), door(D).

{ happens(choosesdoor_by(D, player), 3)                                     : door(D) } = 1.
initiates(choosesdoor_by(D, player), picked(D), T) :- time(T), door(D).

{ happens(choosesdoor_by(D, monty), 4)                                      : door(D) } = 1.
initiates(choosesdoor_by(D, monty), opened(D), T) :- time(T), door(D).

% Monty may only choose doors which are not chosen yet.
:- happens(choosesdoor_by(1, monty), 4), happens(choosesdoor_by(1, gamemaster), 2).
:- happens(choosesdoor_by(2, monty), 4), happens(choosesdoor_by(2, gamemaster), 2).
:- happens(choosesdoor_by(3, monty), 4), happens(choosesdoor_by(3, gamemaster), 2).

:- happens(choosesdoor_by(1, monty), 4), happens(choosesdoor_by(1, player), 3).
:- happens(choosesdoor_by(2, monty), 4), happens(choosesdoor_by(2, player), 3).
:- happens(choosesdoor_by(3, monty), 4), happens(choosesdoor_by(3, player), 3).

{ happens(switch, 5); happens(stay, 5) } = 1.
initiates(switch, switched, 5).
terminates(stay, switched, 5).

% END

won :- maxtime(MT), door(D),
  holdsAt(prizeisbehind(D), MT),
  holdsAt(picked(D),        MT),
  not holdsAt(switched,     MT).
won :- maxtime(MT), door(D1), door(D2),
  D1 != D2,
  holdsAt(prizeisbehind(D1),     MT),
  holdsAt(picked(D2),            MT),
  holdsAt(switched,              MT).
lost :- not won.

% QUERY

% only show models where the player won.
:- lost.

#show happens/2.
#show won/0.
#show lost/0.

"""

In [5]:
solve(event_calculus_axioms + monty)

Answer  1: { happens(choosesdoor_by(2,gamemaster),2) happens(choosesdoor_by(2,player),3) happens(choosesdoor_by(3,monty),4) happens(stay,5) won }.
Answer  2: { happens(choosesdoor_by(2,gamemaster),2) happens(choosesdoor_by(2,player),3) happens(choosesdoor_by(1,monty),4) happens(stay,5) won }.
Answer  3: { happens(choosesdoor_by(3,gamemaster),2) happens(choosesdoor_by(3,player),3) happens(choosesdoor_by(1,monty),4) happens(stay,5) won }.
Answer  4: { happens(choosesdoor_by(3,gamemaster),2) happens(choosesdoor_by(3,player),3) happens(choosesdoor_by(2,monty),4) happens(stay,5) won }.
Answer  5: { happens(choosesdoor_by(1,gamemaster),2) happens(choosesdoor_by(1,player),3) happens(choosesdoor_by(2,monty),4) happens(stay,5) won }.
Answer  6: { happens(choosesdoor_by(1,gamemaster),2) happens(choosesdoor_by(1,player),3) happens(choosesdoor_by(3,monty),4) happens(stay,5) won }.
Answer  7: { happens(choosesdoor_by(3,gamemaster),2) happens(choosesdoor_by(2,player),3) happens(choosesdoor_by(1,mont

<block>:22:3-20: info: atom does not occur in any rule head:
  releases(E,F,T)

<block>:40:3-20: info: atom does not occur in any rule head:
  releases(E,F,T)

<block>:48:3-26: info: atom does not occur in any rule head:
  trajectory(F1,T1,F2,T2)

<block>:56:3-16: info: atom does not occur in any rule head:
  initiallyP(F)

